In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
previous = '/home/bay001/projects/kes_20160307/data/kestrel5-reclustered.no-aerv.no-mtdna.no-vec.no-virus.fasta'
current = '/home/bay001/projects/kes_20160307/data/kestrel5-reclustered.no-aerv.no-mtdna.no-vec.no-virus.no-bac.fasta'
bacterialist = '/home/bay001/projects/kes_20160307/data/bacteria.headers'
blast_file = '/home/bay001/projects/kes_20160307/data/diamond/all.blast'
blast_head = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch',
              'gapopen', 'qstart', 'qend', 'sstart', 'send',
              'evalue', 'bitscore']

In [3]:
bacteria = pd.read_table(bacterialist,names=['bacteria'])
blast = pd.read_table(blast_file,names=blast_head)
blast.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,unmapped-49-contig_list_contig_324-0,UniRef90_UPI000529CB21,99.5,187,1,0,31453,30893,70,256,2.300000e-104,392.1
1,unmapped-49-contig_list_contig_324-0,UniRef90_UPI000529CB21,82.5,80,9,3,36063,35827,1,76,3.100000e-24,125.9
2,unmapped-49-contig_list_contig_324-0,UniRef90_UPI000529CB21,100.0,26,0,0,27759,27682,256,281,2.700000e-04,59.7
3,unmapped-49-contig_list_contig_324-0,UniRef90_E1BYF3,99.5,187,1,0,31453,30893,70,256,2.300000e-104,392.1
4,unmapped-49-contig_list_contig_324-0,UniRef90_E1BYF3,97.5,121,3,0,27759,27397,256,376,2.600000e-60,245.7


In [4]:
print(blast.shape)
blast = blast[blast['evalue'] < 1e-10]
print(blast.shape)
blast['uniref'] = blast['sseqid'].str.replace('UniRef90_','')
blast.sort_values(by='evalue',inplace=True)
blast.drop_duplicates('qseqid',inplace=True)
print(blast.shape)
blast.head()

(3001772, 12)
(1944271, 12)
(60670, 13)


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,uniref
1363364,unmapped-49-contig_list_contig_64936-0,UniRef90_G1KX46,74.5,768,190,3,265,2565,1,763,0.0,1107.0,G1KX46
928192,unmapped-49-contig_list_contig_6250-0,UniRef90_A0A091G0Z8,92.1,1077,76,2,499,3714,1,1073,0.0,1980.7,A0A091G0Z8
75828,unmapped-49-contig_list_contig_29731-0,UniRef90_UPI0005226B71,82.5,1056,185,0,5095,1928,1233,2288,0.0,1697.2,UPI0005226B71
1255090,EC-4AK111_TAGCTT_R1_(paired)_contig_6330-0,UniRef90_V8NBR3,78.4,828,177,2,2712,232,23,849,0.0,1323.1,V8NBR3
1254783,unmapped-49-contig_list_contig_56574-0,UniRef90_UPI000642AD0D,71.4,767,199,6,2794,539,394,1155,0.0,1077.0,UPI000642AD0D


In [5]:
some_series = bacteria['bacteria'].str.extract('>([\w\d]+)')
to_filter_out = pd.DataFrame(some_series)
to_filter_out.head()

/home/bay001/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


,bacteria
0,UniRef90_A5EJ46
1,UniRef90_A4YUJ5
2,UniRef90_Q93AG0
3,UniRef90_Q9I4V0
4,UniRef90_P84529


In [6]:
filter_these_contigs = list(pd.merge(blast,to_filter_out,how='left',left_on='sseqid',right_on='bacteria').dropna()['qseqid'])
print(len(filter_these_contigs))
filter_these_contigs

5


['unmapped-49-contig_list_contig_110476-0',
 'unmapped-49-contig_list_contig_178884-0',
 'unmapped-49-contig_list_contig_37400-0',
 'unmapped-49-contig_list_contig_37401-0',
 'unmapped-49-contig_list_contig_44501-0']

In [7]:
from Bio import SeqIO
records = []
i = 0
handle = open(previous, "rU")
for record in SeqIO.parse(handle, "fasta"):
    if record.id not in filter_these_contigs:
        records.append(record)
    else:
        i = i + 1
        print(i),
        
SeqIO.write(records,current,"fasta")
handle.close()

1 2 3 4 5


In [8]:
! grep '>' $previous | wc -l

284640


In [9]:
! grep '>' $current | wc -l

284635


In [10]:
! grep 'unmapped-49-contig_list_contig_110476-0' $previous

>unmapped-49-contig_list_contig_110476-0
